In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.drop(['RowNumber','CustomerId','Surname'], axis= 'columns', inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f'{column}: {df[column].unique()}')

In [6]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


In [7]:
df1 = pd.get_dummies(data=df, columns=['Geography', 'Gender'])
df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [8]:
print_unique_col_values(df1)

In [9]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [11]:
col_to_scale=['CreditScore', 'Age', 'Tenure','Balance', 'NumOfProducts', 'EstimatedSalary']

In [12]:
df1[col_to_scale] = scaler.fit_transform(df1[col_to_scale])

In [13]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0,1,0
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1,1,0
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0,1,0
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0,1,0
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1,1,0


In [14]:
X = df1.drop('Exited',axis='columns')
y = df1.Exited

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)


In [16]:
X_train.shape

(8000, 13)

In [17]:
X_test.shape

(2000, 13)

In [18]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

In [19]:
def ANN(X_train, y_train, X_test, y_test, loss):
    model = keras.Sequential([
        keras.layers.Dense(13, input_dim=13, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=100)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [20]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5277 - accuracy: 0.7720
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4689 - accuracy: 0.7962
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4522 - accuracy: 0.8016
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4398 - accuracy: 0.8098
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4339 - accuracy: 0.8110
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4303 - accuracy: 0.8120
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4286 - accuracy: 0.8150
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4261 - accuracy: 0.8151
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4249 - accuracy: 0.8163
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4234 - accura

250/250 [==============================] - 0s 1ms/step - loss: 0.3367 - accuracy: 0.8622
Epoch 82/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3366 - accuracy: 0.8630
Epoch 83/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3365 - accuracy: 0.8630
Epoch 84/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3357 - accuracy: 0.8631
Epoch 85/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3356 - accuracy: 0.8621
Epoch 86/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3363 - accuracy: 0.8608
Epoch 87/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3359 - accuracy: 0.8644
Epoch 88/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3367 - accuracy: 0.8621
Epoch 89/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3360 - accuracy: 0.8620
Epoch 90/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3345 - accuracy: 

In [23]:
from sklearn.model_selection import train_test_split
X = df_test_under.drop('Exited', axis='columns')
y = df_test_under['Exited']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [24]:
y_train.value_counts()

1    1630
0    1629
Name: Exited, dtype: int64

In [25]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
102/102 [==============================] - 1s 1ms/step - loss: 0.6860 - accuracy: 0.5545
Epoch 2/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6611 - accuracy: 0.6324
Epoch 3/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6365 - accuracy: 0.6554
Epoch 4/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6231 - accuracy: 0.6600
Epoch 5/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6146 - accuracy: 0.6772
Epoch 6/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6082 - accuracy: 0.6787
Epoch 7/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6025 - accuracy: 0.6803
Epoch 8/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5942 - accuracy: 0.6818
Epoch 9/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5892 - accuracy: 0.6886
Epoch 10/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5822 - accura

102/102 [==============================] - 0s 1ms/step - loss: 0.4518 - accuracy: 0.7828
Epoch 82/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4529 - accuracy: 0.7818
Epoch 83/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4532 - accuracy: 0.7846
Epoch 84/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4544 - accuracy: 0.7806
Epoch 85/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4519 - accuracy: 0.7828
Epoch 86/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4516 - accuracy: 0.7831
Epoch 87/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4532 - accuracy: 0.7824
Epoch 88/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4485 - accuracy: 0.7852
Epoch 89/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4497 - accuracy: 0.7852
Epoch 90/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4515 - accuracy: 

In [30]:
y_pred_final = y_preds.copy()
for i in range(len(y_preds)):
    n_ones = y_pred_final[i] + y_pred_final[i] + y_pred_final[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [31]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.81      0.69      0.75       408
           1       0.73      0.84      0.78       407

    accuracy                           0.76       815
   macro avg       0.77      0.76      0.76       815
weighted avg       0.77      0.76      0.76       815

